In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

portugal_districts = {
    '01': 'Aveiro',
    '02': 'Beja',
    '03': 'Braga',
    '04': 'Bragança',
    '05': 'Castelo Branco',
    '06': 'Coimbra',
    '07': 'Évora',
    '08': 'Faro',
    '09': 'Guarda',
    '10': 'Leiria',
    '11': 'Lisboa',
    '12': 'Portalegre',
    '13': 'Porto',
    '14': 'Santarém',
    '15': 'Setúbal',
    '16': 'Viana do Castelo',
    '17': 'Vila Real',
    '18': 'Viseu',
    '31': 'Madeira',
    '32': 'Madeira',
    '41': 'Açores',
    '42': 'Açores',
    '43': 'Açores',
    '44': 'Açores',
    '45': 'Açores',
    '46': 'Açores',
    '47': 'Açores',
    '48': 'Açores',
    '49': 'Açores'
}

pd.set_option('display.max_rows', 20)

nac21 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2021", converters={'MUNICIPIO':str,'FREGUESIA':str}, skiprows=[0])
nac11 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2011", converters={'MUNICIPIO_CAOP2013':str,'FREGUESIA_CAOP2013':str}, skiprows=[0])
nac01 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2001", converters={'CONCELHO_EU02':str,'FREGUESIA_EU02':str}, skiprows=[0])
tipologias2014 = pd.read_excel('dados/tipologias2014.xlsx').rename(columns={"Código": "FREGUESIA"}).set_index("FREGUESIA")
corr_xlsx = pd.read_excel('dados/corr/correspondencias.xlsx', converters={'cod_old':str,'cod_new':str})
corr = dict(zip(corr_xlsx.cod_old,corr_xlsx.cod_new))
corr_dsg = dict(zip(corr_xlsx.dsg_old,corr_xlsx.dsg_new))

nac21 = nac21[(nac21["FREGUESIA"] != '000000') & (nac21["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac21 = nac21.rename(columns={"Designação CAOP2013": "FF_DSG", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac21 = nac21[["CC_DSG", "FREGUESIA", "FF_DSG", "TOTAL", "PT"]].set_index("FREGUESIA")

nac11 = nac11[(nac11["FREGUESIA_CAOP2013"] != '000000') & (nac11["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac11 = nac11.rename(columns={"Designação CAOP2013": "FF_DSG", "FREGUESIA_CAOP2013": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac11 = nac11[["CC_DSG", "FREGUESIA", "FF_DSG", "TOTAL", "PT"]].set_index("FREGUESIA")

nac01 = nac01[(nac01["FREGUESIA_EU02"] != '000000') & (nac01["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac01 = nac01.rename(columns={"FREGUESIA_EU02": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac01["FREGUESIA"] = nac01["FREGUESIA"].map(corr)
nac01["FF_DSG"] = nac01["FF_DSG"].map(corr_dsg)
nac01 = nac01.groupby('FREGUESIA').agg({'CC_DSG': 'first', "FF_DSG": 'first', 'TOTAL': 'sum', 'PT': 'sum', 'EST': 'sum', 'APA': 'sum'})
nac01 = nac01[["CC_DSG", "FF_DSG", "TOTAL", "PT"]]

nac11 = nac11.drop(['FF_DSG', 'CC_DSG'], axis=1)
nac01 = nac01.drop(['FF_DSG', 'CC_DSG'], axis=1)
nac21 = nac21.rename(columns={"TOTAL":"TOTAL_INE_21", "PT":"PT_INE_21"})
nac11 = nac11.add_suffix('_INE_11')
nac01 = nac01.add_suffix('_INE_01')

tipologias2014 = tipologias2014.drop("Designação", axis=1)
tipologias2014["Distrito"] = tipologias2014.index.map(lambda x: portugal_districts.get(x[:2], 'Unknown'))

# join the dataframes on the index
populacao = tipologias2014.merge(nac21, left_index=True, right_index=True).merge(nac11, left_index=True, right_index=True).merge(nac01, left_index=True, right_index=True)

#MAI
ele21 = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_20211231.xlsx", sheet_name="Freguesia_Consulado", converters={'Código':str, "NAC":int, "UE": int, "ER":int,"TOTAL":int}).rename(columns={"Código":"FREGUESIA"}).set_index("FREGUESIA")
ele11 = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_20111231.xls", sheet_name="Freguesia_Consulado", converters={'Código':str, "NAC":int, "UE": int, "ER":int,"TOTAL":int}).rename(columns={"Código":"FREGUESIA"})
ele11["FREGUESIA"] = ele11["FREGUESIA"].map(corr)
ele11 = ele11[ele11["Nac"]>0]
ele11 = ele11.groupby('FREGUESIA').agg({'Nome': 'first', 'Nac': 'sum', 'UE': 'sum', 'ER': 'sum', 'CNREUE': 'sum', 'CNRE':'sum', "TOTAL":'sum'})

ele21 = ele21.drop(["Distrito/Ilha/Continente > Concelho/País > Freguesia/Consulado", "UE", "ER"], axis=1)
ele21 = ele21.rename(columns={"TOTAL":"TOTAL_MAI_21", "NAC":"PT_MAI_21"})
ele11 = ele11.drop(["Nome", "UE", "ER", "CNREUE", "CNRE"], axis=1)
ele11 = ele11.rename(columns={"TOTAL":"TOTAL_MAI_11", "Nac":"PT_MAI_11"})

mai = ele21.merge(ele11,left_index=True, right_index=True)

#Comparacao
comp = populacao.merge(mai, left_index=True, right_index=True)
comp = comp.drop(["TOTAL_INE_21", "TOTAL_INE_11", "TOTAL_INE_01", "TOTAL_MAI_21", "TOTAL_MAI_11"], axis = 1)

comp["diff_abs_21"] = comp["PT_MAI_21"] - comp["PT_INE_21"]
comp["diff_abs_21"] = comp["diff_abs_21"].astype(float)
comp["diff_abs_11"] = comp["PT_MAI_11"] - comp["PT_INE_11"]
comp["diff_rel_21"] = comp["diff_abs_21"] / comp["PT_INE_21"] * 100
comp["diff_rel_11"] = comp["diff_abs_11"] / comp["PT_INE_11"] * 100
comp['diff_rel_21'] = comp['diff_rel_21'].astype(float).round(1)
comp['diff_rel_11'] = comp['diff_rel_11'].astype(float).round(1)
comp_APU = comp[comp["TIPAU"] == "APU"]
comp_AMU = comp[comp["TIPAU"] == "AMU"]
comp_APR = comp[comp["TIPAU"] == "APR"]

def print_mapa(var="diff_rel_21", ano = 2021):
    district_stats = comp.groupby('Distrito')[var].apply(lambda x: pd.Series({'weighted_mean_diff': (x * comp.loc[x.index, 'PT_INE_21']).sum() / comp.loc[x.index, 'PT_INE_21'].sum()}))
    district_stats = pd.DataFrame(district_stats).reset_index().drop(columns=["level_1"]).set_index("Distrito")

    mapa = gpd.read_file('PRT_adm1.shp')
    mapa = mapa.replace({"Azores":"Açores"})
    mapa = mapa.merge(district_stats, left_on='NAME_1', right_index=True)

    # Create the map
    fig, ax = plt.subplots(figsize=(12, 8))
    mapa.plot(column=var, cmap='OrRd', legend=True, ax=ax)
    ax.set_title('Erro relativo médio por distrito (poderado) ' + str(ano), fontsize=16)
    ax.set_axis_off()

    # Add labels to the map with the average population for each district
    for idx, row in mapa.iterrows():
        plt.annotate(text=f"{row[var]:.0f}", xy=row['geometry'].centroid.coords[0], horizontalalignment='center', fontsize=15)
    plt.show()
    
def print_boxplot(var = "diff_rel_21", titulo = "Erro relativo p/tipo - 2021"):
    data = [comp[var], comp_APU[var], comp_AMU[var], comp_APR[var]]

    # Plot the boxplots
    fig, ax = plt.subplots()
    boxplots = ax.boxplot(data)

    # Add x-axis labels
    ax.set_title(titulo)
    ax.set_xticklabels(['Todas', "APU", "AMU", "APR"])
    ax.set_ylabel("%")
    ax.set_ylim([-80, 180])
    # Show the plot
    plt.show()

In [36]:
cmap = {'APR': 'green', 'APU': 'red', 'AMU': 'blue'}


fig = px.scatter(comp_APU, x='PT_INE_11', y='diff_rel_11', hover_data=['FF_DSG', "PT_MAI_11"])

# Show the plot
fig.show()

In [26]:
comp_APU.head()

,TIPAU,Distrito,CC_DSG,FF_DSG,PT_INE_21,PT_INE_11,PT_INE_01,PT_MAI_21,PT_MAI_11,diff_abs_21,diff_abs_11,diff_rel_21,diff_rel_11
FREGUESIA,,,,,,,,,,,,,
160102,APU,Viana do Castelo,Arcos de Valdevez,Aguiã,615,587,571,801,871,186.0,284,30.2,48.4
160124,APU,Viana do Castelo,Arcos de Valdevez,Paçô,814,754,654,1031,1044,217.0,290,26.7,38.5
160153,APU,Viana do Castelo,Arcos de Valdevez,União das freguesias de Arcos de Valdevez (São...,1362,1300,1313,1711,1826,349.0,526,25.6,40.5
160154,APU,Viana do Castelo,Arcos de Valdevez,União das freguesias de Arcos de Valdevez (Sal...,2215,2018,1699,2704,2459,489.0,441,22.1,21.9
160157,APU,Viana do Castelo,Arcos de Valdevez,União das freguesias de Guilhadeses e Santar,972,1030,893,1190,1173,218.0,143,22.4,13.9


In [107]:
corr_xlsx = pd.read_excel('dados/corr/correspondencias.xlsx', converters={'cod_old':str,'cod_new':str})
nac21 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2021", converters={'MUNICIPIO':str,'FREGUESIA':str}, skiprows=[0])
nac11 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2011", converters={'MUNICIPIO_CAOP2013':str,'FREGUESIA_CAOP2013':str}, skiprows=[0])
nac01 = pd.read_excel('dados/dados_INE.xlsx', sheet_name="Nacionalidade2001", converters={'CONCELHO_EU02':str,'FREGUESIA_EU02':str}, skiprows=[0])
tipologias2014 = pd.read_excel('dados/tipologias2014.xlsx').rename(columns={"Código": "FREGUESIA"}).set_index("FREGUESIA")
corr_xlsx = pd.read_excel('dados/corr/correspondencias.xlsx', converters={'cod_old':str,'cod_new':str})
corr = dict(zip(corr_xlsx.cod_old,corr_xlsx.cod_new))

nac21 = nac21[(nac21["FREGUESIA"] != '000000') & (nac21["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac21 = nac21.rename(columns={"Designação CAOP2013": "FF_DSG", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac21 = nac21[["CC_DSG", "FREGUESIA", "FF_DSG", "TOTAL", "PT"]]
nac21["FREGUESIA"] = nac21["FREGUESIA"].map(corr)

nac11 = nac11[(nac11["FREGUESIA_CAOP2013"] != '000000') & (nac11["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac11 = nac11.rename(columns={"Designação CAOP2013": "FF_DSG", "FREGUESIA_CAOP2013": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac11 = nac11[["CC_DSG", "FREGUESIA", "FF_DSG", "TOTAL", "PT"]]
nac11["FREGUESIA"] = nac11["FREGUESIA"].map(corr)

nac01 = nac01[(nac01["FREGUESIA_EU02"] != '000000') & (nac01["GRUPO ETÁRIO"] == "2 18 ou mais anos")]
nac01 = nac01.rename(columns={"FREGUESIA_EU02": "FREGUESIA", "Portuguesa":"PT", "Estrangeira":"EST", "Apátrida":"APA", "Total":"TOTAL"})
nac01["FREGUESIA"] = nac01["FREGUESIA"].map(corr)

ele21 = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_20211231.xlsx", sheet_name="Freguesia_Consulado", converters={'Código':str, "NAC":int, "UE": int, "ER":int,"TOTAL":int}).rename(columns={"Código":"FREGUESIA"}).set_index("FREGUESIA")
ele11 = pd.read_excel("dados/MAI/BDRE_Contagem_Eleitores_20111231.xls", sheet_name="Freguesia_Consulado", converters={'Código':str, "NAC":int, "UE": int, "ER":int,"TOTAL":int}).rename(columns={"Código":"FREGUESIA"})
ele11["FREGUESIA"] = ele11["FREGUESIA"].map(corr)
ele11 = ele11[ele11["Nac"]>0]
ele11 = ele11.groupby('FREGUESIA').agg({'Nome': 'first', 'Nac': 'sum', 'UE': 'sum', 'ER': 'sum', 'CNREUE': 'sum', 'CNRE':'sum', "TOTAL":'sum'})

ele21 = ele21.drop(["Distrito/Ilha/Continente > Concelho/País > Freguesia/Consulado", "UE", "ER"], axis=1)
ele21 = ele21.rename(columns={"TOTAL":"TOTAL_MAI_21", "NAC":"PT_MAI_21"})
ele11 = ele11.drop(["Nome", "UE", "ER", "CNREUE", "CNRE"], axis=1)
ele11 = ele11.rename(columns={"TOTAL":"TOTAL_MAI_11", "Nac":"PT_MAI_11"})

In [108]:
set(nac21["FREGUESIA"]) - set(nac01["FREGUESIA"])

set()

In [109]:
set(ele11.index) - set(nac01["FREGUESIA"])

set()

,PT_MAI_11,TOTAL_MAI_11
FREGUESIA,,
010103,3694,3694
010109,2850,2852
010112,3129,3134
010119,4461,4463
010121,12379,12386
